<a href="https://colab.research.google.com/github/Esantomi/1930s-kr-lit-subword/blob/main/1930s_kr_lit_subword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인문 데이터 가공 및 근대 국어 서브워드

## dask를 이용한 조선일보 빅데이터 재조합

### 환경 설정

In [ ]:
pip install dask

Looking in indexes: https://opentuna.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import glob
from tqdm import tqdm
import dask.dataframe as dd

In [ ]:
pwd

'C:\\PythonWork\\haein'

### 데이터 불러오기

In [ ]:
df = dd.read_csv("./01_preprocessed_data/chosun_preprocessed.csv", \
                 sep=",", dtype=str, encoding='UTF-8', \
                 on_bad_lines="skip", engine='python')

# the number of partitions
print(f"해당 파일이 분할된 파일 개수는 {df.npartitions}개입니다.")

해당 파일이 분할된 파일 개수는 233개입니다.


In [ ]:
df.head()

,id,page_no,regdate,type,publication_day,mes,section,publication_date,completeness,news_name,...,subtitle_korean,subtitle_archaic_hangul,subtitle_archaic,body_archaic_hangul,body_archaic,body_hanja_korean,body_korean,authors_name,authors_name_hanja,year
0,119200305e10311,3,2020-03-11T18:13:46+0900,smallad,5,e,소형광고,1920-03-05,Y,cs,...,NaN,NaN,NaN,축(祝) …조(朝)；선(蘚) n 보창(報創) 간(刊)\n조(趙)… . … . 명(命...,祝 …朝；蘚 n 報創 刊\n趙… . … . 命· 九\n,NaN,NaN,NaN,NaN,1920
1,119200305e10313,3,2020-03-11T18:13:46+0900,adassemblage,5,e,군집광고,1920-03-05,Y,cs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1920
2,119200305e1044,4,2020-03-11T18:13:47+0900,smallad,5,e,소형광고,1920-03-05,Y,cs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1920
3,119200305e1046,4,2020-03-11T18:13:47+0900,adassemblage,5,e,군집광고,1920-03-05,Y,cs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1920
4,119200305e1032,3,2020-03-31T13:48:20+0900,article,5,e,사회,1920-03-05,Y,cs,...,NaN,NaN,NaN,조선일보(朝鮮日報)\n창간(創刊)을축(祝)함\n후작(候爵) 대외중신(大隈重信)\n조...,朝鮮日報\n創刊을祝함\n候爵 大隈重信\n朝鮮合邦以來歲를閱한지玆에十年이되鮮人의自營하난...,조선합방이 이루어진 지 10년이 되었는데 조선인이 운영하는 신문이 없었기 때문에 지...,조선합방이 이루어진 지 10년이 되었는데 조선인이 운영하는 신문이 없었기 때문에 지...,NaN,NaN,1920


In [ ]:
df.tail()

,id,page_no,regdate,type,publication_day,mes,section,publication_date,completeness,news_name,...,subtitle_korean,subtitle_archaic_hangul,subtitle_archaic,body_archaic_hangul,body_archaic,body_hanja_korean,body_korean,authors_name,authors_name_hanja,year
32947,2456719991231m1345,34,2020-03-18T20:29:57+0900,article,5,m,사회,1999-12-31,Y,cs,...,NaN,40년간 고물 수집 외대 1억 기증후 타계,40년간 고물 수집 외대 1억 기증후 타계,폐품 할머니\n40년간 고물 수집\n외대 1억 기증후 타계\n 지난 7월 장경자(장...,폐품 할머니\n40년간 고물 수집\n외대 1억 기증후 타계\n 지난 7월 장경자(張...,NaN,NaN,정병선,鄭昺善,1999
32948,2456719991231m1355,35,2020-03-18T20:29:59+0900,article,5,m,사회,1999-12-31,Y,cs,...,NaN,상가(商街)주인 할머니,商街주인 할머니,고시준비생 장학금\n외대에 3억원 전달\n상가(商街)주인 할머니\n 지난 14일힐튼...,고시준비생 장학금\n외대에 3억원 전달\n商街주인 할머니\n 지난 14일힐튼호텔 컨...,NaN,NaN,정병선,鄭昺善,1999
32949,2456719991231m1358,35,2020-03-18T20:29:59+0900,article,5,m,사회,1999-12-31,Y,cs,...,NaN,수원지법 컴퓨터‘Y2K’\n피고인에 엉뚱한 소환장,수원지법 컴퓨터‘Y2K’\n피고인에 엉뚱한 소환장,“1900년에 출두하시오”\n수원지법 컴퓨터‘Y2K’\n피고인에 엉뚱한 소환장\n ...,“1900년에 출두하시오”\n수원지법 컴퓨터‘Y2K’\n피고인에 엉뚱한 소환장\n ...,NaN,NaN,권상은,權祥銀,1999
32950,2456719991231m1349,34,2020-03-18T20:29:57+0900,photo,5,m,사회,1999-12-31,Y,cs,...,NaN,NaN,NaN,◇고(故)장경자씨\n,◇故장경자씨\n,NaN,NaN,NaN,NaN,1999
32951,2456719991231m1354,35,2020-03-18T20:29:59+0900,article,5,m,사회,1999-12-31,Y,cs,...,NaN,콩나물 할머니,콩나물 할머니,콩나물 장사 30여년\n충북대 12억건물 기증\n콩나물 할머니\n “모든 걸주고 나...,콩나물 장사 30여년\n충북대 12억건물 기증\n콩나물 할머니\n “모든 걸주고 나...,NaN,NaN,유태종,劉泰鍾,1999


### 1930년대 문학 텍스트(시, 소설) 추출

In [ ]:
filter_criteria_chosun = (
    (df['type'] == 'novel')&(df['year'] == '1930') |
    (df['type'] == 'poem')&(df['year'] == '1930') |
    (df['type'] == 'novel')&(df['year'] == '1931') |
    (df['type'] == 'poem')&(df['year'] == '1931') |
    (df['type'] == 'novel')&(df['year'] == '1932') |
    (df['type'] == 'poem')&(df['year'] == '1932') |
    (df['type'] == 'novel')&(df['year'] == '1933') |
    (df['type'] == 'poem')&(df['year'] == '1933') |
    (df['type'] == 'novel')&(df['year'] == '1934') |
    (df['type'] == 'poem')&(df['year'] == '1934') |
    (df['type'] == 'novel')&(df['year'] == '1935') |
    (df['type'] == 'poem')&(df['year'] == '1935') |
    (df['type'] == 'novel')&(df['year'] == '1936') |
    (df['type'] == 'poem')&(df['year'] == '1936') |
    (df['type'] == 'novel')&(df['year'] == '1937') |
    (df['type'] == 'poem')&(df['year'] == '1937') |
    (df['type'] == 'novel')&(df['year'] == '1938') |
    (df['type'] == 'poem')&(df['year'] == '1938') |
    (df['type'] == 'novel')&(df['year'] == '1939') |
    (df['type'] == 'poem')&(df['year'] == '1939')
)

In [ ]:
chosun_lit_df = df.loc[filter_criteria_chosun]

In [ ]:
chosun_lit_df.head(npartitions=-1)

,id,page_no,regdate,type,publication_day,mes,section,publication_date,completeness,news_name,...,subtitle_korean,subtitle_archaic_hangul,subtitle_archaic,body_archaic_hangul,body_archaic,body_hanja_korean,body_korean,authors_name,authors_name_hanja,year
10339,322719300101e1151,15,2020-03-31T12:08:25+0900,poem,3,e,기타,1930-01-01,Y,cs,...,NaN,NaN,NaN,해는또다시밧귀거니\n김안서(金岸曙)\n별\n비오다 바람개인 어둡은밤날\n끈임업시 흘...,해는또다시밧귀거니\n金岸曙\n별\n비오다 바람개인 어둡은밤날\n끈임업시 흘으는 맑은...,김안서(金岸曙)</br></br> 별</br></br> 비오다 바람개인 어둡은 밤날...,김안서</br></br> 별</br></br> 비오다 바람개인 어둡은 밤날</br>...,金岸曙,金岸曙,1930
10399,322719300101e1134,13,2020-03-31T12:08:23+0900,novel,3,e,기타,1930-01-01,Y,cs,...,밀담,밀담(일(一)),밀담(一),광분(狂奔)\n【77】\n밀담(일(一))\n염상섭작(廉想涉作)\n안석영화(安夕影画)...,狂奔\n【77】\n밀담(一)\n廉想涉作\n安夕影画\n 숙뎡이는 정방이가 간뒤에혼자 ...,밀담(一)</br></br> 염상섭작(廉想涉作)</br></br> 안석영화(安夕影...,밀담</br></br> 염상섭작</br></br> 안석영화</br></br> 숙정...,廉想涉,廉想涉,1930
10446,322819300102e1034,3,2020-03-31T12:08:42+0900,novel,4,e,기타,1930-01-02,Y,cs,...,NaN,NaN,NaN,명일(明日)\n한수암(韓水岩)\n란해는 사감선생의 특별허락을어더가시고 밤여듧시에 긔...,明日\n韓水岩\n란해는 사감선생의 특별허락을어더가시고 밤여듧시에 긔숙사대문을나섯다 ...,한수암(韓水岩)</br></br> 란해는 사감 선생의 특별 허락을 얻어가시고 밤 여...,한수암</br></br> 란해는 사감 선생의 특별 허락을 얻어가시고 밤 여듧시에 기...,韓水岩,韓水岩,1930
10458,322819300102e1119,11,2020-03-31T12:08:38+0900,novel,4,e,기타,1930-01-02,Y,cs,...,밀담,밀담(일(一)),밀담(一),광분(狂奔)\n【79】\n염상섭작(廉想涉作)\n안석영화(安夕影画)\n밀담(일(一))...,狂奔\n【79】\n廉想涉作\n安夕影画\n밀담(一)\n 원량이가 시퉁거리는눈치를보자 ...,염상섭작(廉想涉作)</br></br> 안석영화(安夕影画)</br></br> 밀담(一...,염상섭작</br></br> 안석영화</br></br> 밀담</br></br> 원량이...,廉想涉,廉想涉,1930
10480,322819300102e1084,8,2020-03-31T12:08:47+0900,novel,4,e,기타,1930-01-02,Y,cs,...,『연애편』,『연애편(戀愛篇)』,『戀愛篇』,작창(作創)\n순정(純情)\n【전승(前承)】\n『연애편(戀愛篇)』\n김동인\n 어언...,作創\n純情\n【前承】\n『戀愛篇』\n김동인\n 어언간 십년도지낫다 지나고보니 꿈결...,『연애편(戀愛篇)』</br></br> 김동인</br></br> 어언간 십 년도 지...,『연애편』</br></br> 김동인</br></br> 어언간 십 년도 지낫다 지나...,김동인,NaN,1930


### dd to pd (dask를 pandas로 변환)

In [ ]:
chosun_lit_pdf = chosun_lit_df.compute()

In [ ]:
chosun_lit_pdf.head()

,id,page_no,regdate,type,publication_day,mes,section,publication_date,completeness,news_name,...,subtitle_korean,subtitle_archaic_hangul,subtitle_archaic,body_archaic_hangul,body_archaic,body_hanja_korean,body_korean,authors_name,authors_name_hanja,year
10339,322719300101e1151,15,2020-03-31T12:08:25+0900,poem,3,e,기타,1930-01-01,Y,cs,...,NaN,NaN,NaN,해는또다시밧귀거니\n김안서(金岸曙)\n별\n비오다 바람개인 어둡은밤날\n끈임업시 흘...,해는또다시밧귀거니\n金岸曙\n별\n비오다 바람개인 어둡은밤날\n끈임업시 흘으는 맑은...,김안서(金岸曙)</br></br> 별</br></br> 비오다 바람개인 어둡은 밤날...,김안서</br></br> 별</br></br> 비오다 바람개인 어둡은 밤날</br>...,金岸曙,金岸曙,1930
10399,322719300101e1134,13,2020-03-31T12:08:23+0900,novel,3,e,기타,1930-01-01,Y,cs,...,밀담,밀담(일(一)),밀담(一),광분(狂奔)\n【77】\n밀담(일(一))\n염상섭작(廉想涉作)\n안석영화(安夕影画)...,狂奔\n【77】\n밀담(一)\n廉想涉作\n安夕影画\n 숙뎡이는 정방이가 간뒤에혼자 ...,밀담(一)</br></br> 염상섭작(廉想涉作)</br></br> 안석영화(安夕影...,밀담</br></br> 염상섭작</br></br> 안석영화</br></br> 숙정...,廉想涉,廉想涉,1930
10446,322819300102e1034,3,2020-03-31T12:08:42+0900,novel,4,e,기타,1930-01-02,Y,cs,...,NaN,NaN,NaN,명일(明日)\n한수암(韓水岩)\n란해는 사감선생의 특별허락을어더가시고 밤여듧시에 긔...,明日\n韓水岩\n란해는 사감선생의 특별허락을어더가시고 밤여듧시에 긔숙사대문을나섯다 ...,한수암(韓水岩)</br></br> 란해는 사감 선생의 특별 허락을 얻어가시고 밤 여...,한수암</br></br> 란해는 사감 선생의 특별 허락을 얻어가시고 밤 여듧시에 기...,韓水岩,韓水岩,1930
10458,322819300102e1119,11,2020-03-31T12:08:38+0900,novel,4,e,기타,1930-01-02,Y,cs,...,밀담,밀담(일(一)),밀담(一),광분(狂奔)\n【79】\n염상섭작(廉想涉作)\n안석영화(安夕影画)\n밀담(일(一))...,狂奔\n【79】\n廉想涉作\n安夕影画\n밀담(一)\n 원량이가 시퉁거리는눈치를보자 ...,염상섭작(廉想涉作)</br></br> 안석영화(安夕影画)</br></br> 밀담(一...,염상섭작</br></br> 안석영화</br></br> 밀담</br></br> 원량이...,廉想涉,廉想涉,1930
10480,322819300102e1084,8,2020-03-31T12:08:47+0900,novel,4,e,기타,1930-01-02,Y,cs,...,『연애편』,『연애편(戀愛篇)』,『戀愛篇』,작창(作創)\n순정(純情)\n【전승(前承)】\n『연애편(戀愛篇)』\n김동인\n 어언...,作創\n純情\n【前承】\n『戀愛篇』\n김동인\n 어언간 십년도지낫다 지나고보니 꿈결...,『연애편(戀愛篇)』</br></br> 김동인</br></br> 어언간 십 년도 지...,『연애편』</br></br> 김동인</br></br> 어언간 십 년도 지낫다 지나...,김동인,NaN,1930


In [ ]:
chosun_lit_pdf.tail()

,id,page_no,regdate,type,publication_day,mes,section,publication_date,completeness,news_name,...,subtitle_korean,subtitle_archaic_hangul,subtitle_archaic,body_archaic_hangul,body_archaic,body_hanja_korean,body_korean,authors_name,authors_name_hanja,year
2532,669719391227e10424,4,2020-04-02T20:35:57+0900,novel,3,e,기타,1939-12-27,Y,cs,...,조조의 복수전,조조(曹操)의복수전(復讐戰)(삼(三)),曹操의復讐戰(三),삼국지(三國志)\n(53)\n조조(曹操)의복수전(復讐戰)(삼(三))\n한(韓) 용(...,三國志\n(53)\n曹操의復讐戰(三)\n韓 龍 雲 譯\n熊 超 畵\n 조조의 군사가...,조조(曹操)의 복수전(復讐戰)(三)</br></br> 한(韓) 용(龍) 운(雲) ...,조조의 복수전</br></br> 한 용 운 역</br></br> 웅 초 화</br...,韓龍雲,韓龍雲,1939
2564,669819391228m10411,4,2020-04-02T20:36:41+0900,novel,4,m,기타,1939-12-28,Y,cs,...,물고기의 근하신년,물고기의근하신년(謹賀新年) (일(一)),물고기의謹賀新年 (一),장편소설(長篇小說)\n사랑의수족관(水族舘)\n(115】\n물고기의근하신년(謹賀新年)...,長篇小說\n사랑의水族舘\n(115】\n물고기의謹賀新年(一)\n金 南 天 作\n鄭 玄...,장편소설(長篇小說)</br></br> 사랑의 수족관(水族舘)</br></br> (1...,장편소설</br></br> 사랑의 수족관</br></br> (115】</br></b...,金南天,金南天,1939
2625,669819391228e10416,4,2020-04-02T20:36:26+0900,novel,4,e,기타,1939-12-28,Y,cs,...,조조의 복수전,조조(曹操)의복수전(復讐戰) (사(四)),曹操의復讐戰 (四),삼국지(三國志)\n(54)\n조조(曹操)의복수전(復讐戰)(사(四))\n한(韓) 용(...,三國志\n(54)\n曹操의復讐戰(四)\n韓 龍 雲 譯\n熊 超 畵\n 이응이 그말...,조조(曹操)의 복수전(復讐戰) (四)</br></br> 한(韓) 용(龍) 운(雲)...,조조의 복수전 </br></br> 한 용 운 역</br></br> 웅 초 화</b...,韓龍雲,韓龍雲,1939
2756,669919391229e10317,3,2020-04-02T20:36:52+0900,novel,5,e,기타,1939-12-29,Y,cs,...,조조의 복수전,조조(曹操)의복수전(復讐戰)(오(五)),曹操의復讐戰(五),삼국지(三國志)(55)\n조조(曹操)의복수전(復讐戰)(오(五))\n한(韓) 용(龍)...,三國志(55)\n曹操의復讐戰(五)\n韓 龍 雲 譯\n熊 超 畵\n 공융이 편지를써...,조조(曹操)의 복수전(復讐戰)(五)</br></br> 한(韓) 용(龍) 운(雲) ...,조조의 복수전</br></br> 한 용 운 역</br></br> 웅 초 화</br...,韓龍雲,韓龍雲,1939
2853,670019391230e1044,4,2020-04-02T20:37:03+0900,novel,6,e,기타,1939-12-30,Y,cs,...,동심(9),동심(童心)(구(九)),童心(九),만주개척민소설(滿洲開拓民小說)\n대지(大地)의아들\n(56)\n이(李) 기(箕) 영...,滿洲開拓民小說\n大地의아들\n(56)\n李 箕 永 作\n崔 木 朗 畵\n童心(九)\...,(56)</br></br> 이(李) 기(箕) 영(永) 작(作)</br></br> 최...,(56)</br></br> 이 기 영 작</br></br> 최 목 낭 화</br><...,李箕永,李箕永,1939


### csv 파일로 저장

In [ ]:
chosun_lit_pdf.to_csv("./02_processed_data/chosun_literature.csv", encoding="utf-8-sig")

## 근대 한국어 서브워드 학습

### 서브워드 학습 준비

In [ ]:
# kiwi 설치
!pip install -U kiwipiepy

Looking in indexes: https://opentuna.cn/pypi/web/simple


In [ ]:
import pandas as pd

# kiwi 로드
from kiwipiepy import Kiwi
from kiwipiepy.sw_tokenizer import SwTokenizer
from kiwipiepy.sw_tokenizer import SwTokenizerConfig

In [ ]:
kiwi = Kiwi()

In [ ]:
# 조선일보 1930-1939 문학 텍스트 데이터 로드
df = pd.read_csv('./02_processed_data/chosun_literature_1930-1939.csv')

df.head()

,Unnamed: 0,id,page_no,regdate,type,publication_day,mes,section,publication_date,completeness,...,subtitle_korean,subtitle_archaic_hangul,subtitle_archaic,body_archaic_hangul,body_archaic,body_hanja_korean,body_korean,authors_name,authors_name_hanja,year
0,10339,322719300101e1151,15,2020-03-31T12:08:25+0900,poem,3,e,기타,1930-01-01,Y,...,NaN,NaN,NaN,해는또다시밧귀거니\n김안서(金岸曙)\n별\n비오다 바람개인 어둡은밤날\n끈임업시 흘...,해는또다시밧귀거니\n金岸曙\n별\n비오다 바람개인 어둡은밤날\n끈임업시 흘으는 맑은...,김안서(金岸曙)</br></br> 별</br></br> 비오다 바람개인 어둡은 밤날...,김안서</br></br> 별</br></br> 비오다 바람개인 어둡은 밤날</br>...,金岸曙,金岸曙,1930
1,10399,322719300101e1134,13,2020-03-31T12:08:23+0900,novel,3,e,기타,1930-01-01,Y,...,밀담,밀담(일(一)),밀담(一),광분(狂奔)\n【77】\n밀담(일(一))\n염상섭작(廉想涉作)\n안석영화(安夕影画)...,狂奔\n【77】\n밀담(一)\n廉想涉作\n安夕影画\n 숙뎡이는 정방이가 간뒤에혼자 ...,밀담(一)</br></br> 염상섭작(廉想涉作)</br></br> 안석영화(安夕影...,밀담</br></br> 염상섭작</br></br> 안석영화</br></br> 숙정...,廉想涉,廉想涉,1930
2,10446,322819300102e1034,3,2020-03-31T12:08:42+0900,novel,4,e,기타,1930-01-02,Y,...,NaN,NaN,NaN,명일(明日)\n한수암(韓水岩)\n란해는 사감선생의 특별허락을어더가시고 밤여듧시에 긔...,明日\n韓水岩\n란해는 사감선생의 특별허락을어더가시고 밤여듧시에 긔숙사대문을나섯다 ...,한수암(韓水岩)</br></br> 란해는 사감 선생의 특별 허락을 얻어가시고 밤 여...,한수암</br></br> 란해는 사감 선생의 특별 허락을 얻어가시고 밤 여듧시에 기...,韓水岩,韓水岩,1930
3,10458,322819300102e1119,11,2020-03-31T12:08:38+0900,novel,4,e,기타,1930-01-02,Y,...,밀담,밀담(일(一)),밀담(一),광분(狂奔)\n【79】\n염상섭작(廉想涉作)\n안석영화(安夕影画)\n밀담(일(一))...,狂奔\n【79】\n廉想涉作\n安夕影画\n밀담(一)\n 원량이가 시퉁거리는눈치를보자 ...,염상섭작(廉想涉作)</br></br> 안석영화(安夕影画)</br></br> 밀담(一...,염상섭작</br></br> 안석영화</br></br> 밀담</br></br> 원량이...,廉想涉,廉想涉,1930
4,10480,322819300102e1084,8,2020-03-31T12:08:47+0900,novel,4,e,기타,1930-01-02,Y,...,『연애편』,『연애편(戀愛篇)』,『戀愛篇』,작창(作創)\n순정(純情)\n【전승(前承)】\n『연애편(戀愛篇)』\n김동인\n 어언...,作創\n純情\n【前承】\n『戀愛篇』\n김동인\n 어언간 십년도지낫다 지나고보니 꿈결...,『연애편(戀愛篇)』</br></br> 김동인</br></br> 어언간 십 년도 지...,『연애편』</br></br> 김동인</br></br> 어언간 십 년도 지낫다 지나...,김동인,NaN,1930


In [ ]:
df_nan_deleted = df.dropna(subset=['body_archaic'], how='any', axis=0)

In [ ]:
df_nan_deleted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12034 entries, 0 to 12037
Data columns (total 45 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               12034 non-null  int64  
 1   id                       12034 non-null  object 
 2   page_no                  12034 non-null  int64  
 3   regdate                  12034 non-null  object 
 4   type                     12034 non-null  object 
 5   publication_day          12034 non-null  int64  
 6   mes                      12034 non-null  object 
 7   section                  12034 non-null  object 
 8   publication_date         12034 non-null  object 
 9   completeness             12034 non-null  object 
 10  news_name                12034 non-null  object 
 11  publication_no           12034 non-null  float64
 12  node_id                  12034 non-null  int64  
 13  source_image_file        12034 non-null  object 
 14  @timestamp            

### 서브워드 학습

In [ ]:
# https://bab2min.github.io/kiwipiepy/v0.15.2/kr/sw_tokenizer.html#kiwipiepy.sw_tokenizer.SwTokenizerConfig
config = SwTokenizerConfig(split_chinese=False) # split_chinese 제외하고 모두 기본 옵션 사용

In [ ]:
# 학습할 데이터를 리스트 형태로 변경
# 문장분리가 필요한가? https://github.com/bab2min/kiwipiepy/issues/132

sentence_list = df_nan_deleted['body_archaic'].tolist()
sentence_list[0]

'해는또다시밧귀거니\n金岸曙\n별\n비오다 바람개인 어둡은밤날\n끈임업시 흘으는 맑은江물은\n제노래에醉하야 결이 가늘제\n별들은 고히어려 반듯기리고.\n어둡다 어둡다며 오가는行客\n얼마나 모혀들어 별을낙다가\n얼마나 돌아섯노 낙시던지고\n덧업는밤 스스로동터밝을제.\n어느벗에게\n(二首)\n歲月은 홀으는물 끗을몰으고\n사람은 한갓되히 나고죽는걸.\n눈오다 바람개인 이아닌밤에\n무어라,맘을일코 설어우는가.\n◇\n밤마다 찬자리에 혼자어든꿈\n깨여보니,煙氣는 슬어지는걸.\n銀河水맑은물에 떠가는배를\n어이나 멈줄는고 흐리운못에.\n——一九二九一二二六——\n'

In [ ]:
# 학습

# save_path = 토크나이저.json 파일 저장 위치
# texts = 학습할 데이터(리스트)
# vocab_size = 단어수
# config = 학습 옵션

tokenizer = SwTokenizer.train(save_path='./tokenizer.json', texts=sentence_list, vocab_size=1000000, config=config)




Tokenizing:   0%|                                                                            | 0/12034 [00:00<?, ?it/s]


Tokenizing:   2%|█▏                                                              | 224/12034 [00:00<00:05, 2208.53it/s]


Tokenizing:   4%|██▋                                                             | 496/12034 [00:00<00:04, 2506.26it/s]


Tokenizing:   6%|███▉                                                            | 752/12034 [00:00<00:05, 2119.14it/s]


Tokenizing:   8%|█████▏                                                          | 976/12034 [00:00<00:05, 2149.22it/s]


Tokenizing:  10%|██████▎                                                        | 1200/12034 [00:00<00:05, 1890.04it/s]


Tokenizing:  12%|███████▎                                                       | 1408/12034 [00:00<00:05, 1869.20it/s]


Tokenizing:  13%|████████▍                                                      | 1600/12034 [00:00<00:06, 1680.45it/s]


Tokenizing:  15%|████

Tokenizing:  79%|██████████████████████████████████████████████████             | 9552/12034 [00:07<00:01, 1329.40it/s]


Tokenizing:  81%|██████████████████████████████████████████████████▊            | 9696/12034 [00:07<00:01, 1253.56it/s]


Tokenizing:  82%|███████████████████████████████████████████████████▍           | 9824/12034 [00:07<00:01, 1190.42it/s]


Tokenizing:  83%|████████████████████████████████████████████████████▎          | 9984/12034 [00:07<00:01, 1295.84it/s]


Tokenizing:  84%|████████████████████████████████████████████████████▏         | 10128/12034 [00:07<00:01, 1329.95it/s]


Tokenizing:  85%|████████████████████████████████████████████████████▉         | 10272/12034 [00:07<00:01, 1227.91it/s]


Tokenizing:  86%|█████████████████████████████████████████████████████▌        | 10400/12034 [00:07<00:01, 1199.34it/s]


Tokenizing:  88%|██████████████████████████████████████████████████████▎       | 10544/12034 [00:08<00:01, 1258.99it/s]


Tokenizing:  89%|███████

Iteration: 0 VocabSize: 159929 Loss: -7.3995
Iteration: 1 VocabSize: 146247 Loss: -7.3508
Finished. Iteration: 1 VocabSize: 146247 Loss: -7.3508


In [ ]:
# 서브워드 토크나이저 성능 확인
print([tokenizer.id2vocab[i] for i in tokenizer.encode(sentence_list[0])])

['해는', '##또다시', '##밧귀', '##거니', '金', '##岸', '##曙', '별', '비', '오/V', '다/E', '바람', '##개인', '어둡/V-I', '은/E', '##밤', '##날', '끈임업시', '흘으', '는/J', '맑/V', '은/E', '##江물', '은/J', '저', '의/J', '##노래', '에/J', '##醉', '하/V', '야/E', '결', '이/J', '가/V', '어/E', '##늘저', '의/J', '별', '들/XSN', '은/J', '고히', '##어려', '반듯', '기리/V', '고/E', '.', '어둡/V-I', '다/E', '어둡/V-I', '다며/E', '오가/V', '는/E', '##行', '##客', '얼마나', '모혀', '들/V', '어/E', '별', '을/J', '##낙', '다그/V', '어/E', '얼마나', '돌아서/V', '엇/E', '노/E', '낙시', '던지/V', '고/E', '덧', '업/V', '는/E', '##밤', '스스로', '##동', '##터', '밝/V', '을/E', '##제', '.', '어느', '##벗', '에게/J', '(', '##二', '##首', ')', '歲月', '은/J', '홀으', '는/J', '##물', '끗', '을/J', '몰/V', '으고/E', '사람', '은/J', '한갓', '되/V', '히/E', '나/V', '고/E', '죽/V', '는/E', '##거', 'ᆯ/J', '.', '눈', '오/V', '다/E', '바람', '##개인', '이/J', '아니/V', 'ᆫ/E', '##밤', '에/J', '무어', '이/V', '라/E', ',', '##맘', '을/J', '##일코', '설/V', '어/E', '울/V', '는가/E', '.', '◇', '밤', '마다/J', '차/V', 'ᆫ/E', '##자리', '에/J', '혼자', '어/E', '들/V', 'ᆫ/E', '##꿈', '깨이/V', '어/E', 

## 이상 소설 텍스트 WordPiece

### 환경 설정

In [ ]:
# kiwi 설치

!pip install -U kiwipiepy

In [ ]:
import pandas as pd

# kiwi 로드
from kiwipiepy import Kiwi
from kiwipiepy.sw_tokenizer import SwTokenizer

In [ ]:
kiwi = Kiwi()

In [ ]:
# 토크나이저 설정
tokenizer = SwTokenizer('./1930s_ko_lit_tokenizer.json', kiwi=kiwi)

### 토크나이저 테스트

In [ ]:
# 이상 날개 중 일부
tokenizer.encode('肉身이흐느적흐느적하도록 疲勞했을때만 情身이 銀貨처럼 맑소')

array([ 57880,      4, 102365, 102365,      1,    277,  47733,      1,
          945,    129,     89,     45,   8240,   5559,      4,  19550,
        25188,    180,   1137,    224], dtype=uint32)

In [ ]:
# 토큰나이징 결과 확인 (subword + 한국어 형태소)
vocab_list = tokenizer.encode('肉身이흐느적흐느적하도록 疲勞했을때만 情身이 銀貨처럼 맑소')
[tokenizer.id2vocab[i] for i in vocab_list]

['##肉身',
 '이/J',
 '##흐느적',
 '##흐느적',
 '하/V',
 '도록/E',
 '疲勞',
 '하/V',
 '었/E',
 '을/E',
 '##때',
 '만/J',
 '情',
 '##身',
 '이/J',
 '銀',
 '##貨',
 '처럼/J',
 '맑/V',
 '소/E']